In [47]:
import netCDF4 as nc
import glob
import numpy as np
fs=glob.glob("output/subset_*.nc")
fs=sorted(fs)
y_lastbin=[]

x_data=[]
y_data=[]
p_type_data=[]
n_seq_data=[]
#print(fs)
y_nonz=[[]for k in range(44+18)]
x_nonz=[[]for k in range(44+18)]
xL=[]
yL=[]
diffL=[]
for f in fs:
    with nc.Dataset(f) as fh:
        bin_nodes=fh.variables['bin_nodes'][:]
        diffL.extend(bin_nodes[:,-1]-(bin_nodes[:,2]-2))

print(np.array(diffL).min(),np.array(diffL).max())


-22 22


In [48]:
import matplotlib.pyplot as plt
#h=plt.hist(diffL,bins=20)
print(len(fs))

645


In [74]:
n_files=len(fs)
x_data=[]
y_data=[]
x_pia_data=[]
y_pia_data=[]
p_type_data=[]
n_seq_data=[]
bin_data=[]
z_meas2L=[]
bin_nodesL=[]
pia_dataL=[]
import tqdm
for f in tqdm.tqdm(fs[:n_files//2]):
    with nc.Dataset(f) as fh:
        
        vars=fh.variables.keys()
        z_meas=fh.variables['z_meas'][:]
        p_type=fh.variables['p_type'][:]
        bin_nodes=fh.variables['bin_nodes'][:]
        sfc_bin=fh.variables['sfc_bin'][:]
        p_rate_cmb=fh.variables['p_rate_cmb'][:]
        dm=fh.variables['dm'][:]
        bin_zero_deg=fh.variables['bin_zero_deg'][:]
        bin_storm_top=fh.variables['bin_storm_top'][:]
        surface_type=fh.variables['surface_type'][:]
        pia=fh.variables['pia'][:]
        z_meas2=np.log10(0.5*10**(z_meas[:,::2]/10)+0.5*10**(z_meas[:,1::2]/10)+1e-9)*10
        z_meas2[z_meas2<0]=0
        for i,z_meas_1d in enumerate(z_meas2):
            if surface_type[i]==0:
                continue
            if p_type[i]==3:
                continue
            itop=bin_storm_top[i]//2
            x_1d=[]
            y_1d=[]
            i_valid=0
            bin_1d=[]
            for k1 in range(bin_nodes[i,0],bin_nodes[i,-1]): 
                x_1d.append([z_meas_1d[k1],k1-bin_nodes[i,2]])
                y_1d.append([np.log10(1+p_rate_cmb[i,k1]/0.1),dm[i,k1]])
                bin_1d.append(k1-bin_nodes[i,2])
                i_valid+=1
            
            if(i_valid>0):
                n_seq_data.append(i_valid)
                bin_data.extend(bin_1d)
                x_data.append(x_1d)
                k1=bin_nodes[i,-1]
                x_pia_data.append([sfc_bin[i,0]-bin_nodes[i,-1]])
                y_pia_data.append([pia[i,0],np.log10(1+p_rate_cmb[i,k1]/0.1),dm[i,k1]])
                y_data.append(y_1d)
                p_type_data.append(p_type[i])
                z_meas2L.append(z_meas_1d.copy())
                bin_nodesL.append(bin_nodes[i,:])
    #break

#vars='z_meas', 'p_type', 'bin_nodes', 'sfc_bin', 'p_rate_cmb', 'dm', 'bin_zero_deg', 'bin_storm_top'


100%|██████████| 322/322 [00:54<00:00,  5.88it/s]


In [75]:
print(len(x_data))
print(np.max(n_seq_data))

254344
70


In [76]:
#plt.plot(x_data[25000][:,0])
n_all=len(x_data)
print(n_all)
x_data_unpacked=np.zeros((n_all,72,2),np.float32)
y_data_unpacked=np.zeros((n_all,72,2),np.float32)
z_meas2L=np.array(z_meas2L)
bin_nodesL=np.array(bin_nodesL)
x_pia_data=np.array(x_pia_data)
y_pia_data=np.array(y_pia_data)
for i in range(n_all):
    nz=len(x_data[i])
    x_data_unpacked[i,:nz,:]=np.array(x_data[i])
    x_data_unpacked[i,:nz,:1]=(x_data_unpacked[i,:nz,:1]-12)/8
    x_data_unpacked[i,:nz,1:2]/=8.0
    y_data_unpacked[i,:nz,:]=np.array(y_data[i])

print(x_pia_data.shape) 


254344
(254344, 1)


In [6]:
import numpy as np
ix=np.random.randint(0,n_all)
#ix=3000
print(z_meas2L.shape,n_all)

for k in range(bin_nodesL[ix,0],bin_nodesL[ix,-1]):
    x1=[z_meas2L[ix][k],k-bin_nodesL[ix,2]]
    x1[0]=(x1[0]-12)/8
    x1[1]=x1[1]/8
    xL.append(x1)
    print(np.array(x1),x_data_unpacked[ix,k-bin_nodesL[ix,0],:])

(663803, 88) 663803
[ 0.17286491 -0.875     ] [ 0.17286491 -0.875     ]
[ 0.37375569 -0.75      ] [ 0.3737557 -0.75     ]
[ 0.06635046 -0.625     ] [ 0.06635046 -0.625     ]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class LSTMPackedModel(nn.Module):
    def __init__(self, input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp):
        super(LSTMPackedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Final output layer
        self.sec_out_fc1=nn.Linear(second_input_size+hidden_size,hidden_size)
        self.sec_out_fc2=nn.Linear(hidden_size,(n_comp+2)*second_output_size)
    def forward(self, x, lengths, x2):
        # Pack the padded sequence
        packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        # LSTM forward pass
        packed_output, (hn, cn) = self.lstm(packed_input)
        #print(packed_output.data.shape)

        # Unpack the sequence
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
        #print(output.shape)
        n_batch = len(lengths)
        batch_indices = torch.arange(n_batch)
        last_output = output[batch_indices,lengths-1,:]  
        last_output=torch.cat([last_output,x2],dim=1)
        # Apply the output layer to the LSTM output
        output_1 = self.fc(output)
        hid1=self.sec_out_fc1(last_output)
        # apply relu
        hid1=torch.relu(hid1)
        output_2 = self.sec_out_fc2(hid1)

        return output_1, output_2


# define an ensemble of LSTMPackedModel models 


# Example usage:
# Assume input is a padded batch with variable sequence lengths
i_test=0
if i_test==1:
    batch_size = 4
    input_size = 2
    second_input_size = 1
    second_output_size = 3
    hidden_size = 20
    num_layers = 2
    output_size = 2
    n_comp=2
    seq_lengths = torch.tensor([41, 21, 12, 33]) 
    padded_input = torch.randn(batch_size, seq_lengths.max(), input_size)
    second_input = torch.randn(batch_size,second_input_size) # Variable lengths of the sequences
    input_size = 2


    model = LSTMPackedModel(input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp)

    padded_input = torch.tensor(x_data_unpacked[:10,:,:])
    seq_lengths = torch.tensor(n_seq_data[:10])
    second_input = torch.tensor(x_pia_data[:10])


#    Forward pass   
    output_1, output_2 = model(padded_input, seq_lengths, second_input)

    print(output_1.shape)
    print(output_2.shape)

from torch.distributions.lowrank_multivariate_normal import LowRankMultivariateNormal
def NLLLoss(output, target, sec_out_size):
    nb, n2 = output.shape
    n1=sec_out_size
    ncomp=n2//sec_out_size-2
    mean = output[:,0:n1]
    cov_diag = torch.exp(output[:,n1:2*n1])+0.01
    factors = output[:,2*n1:].reshape(nb, n1, ncomp)
    dist = LowRankMultivariateNormal(mean, factors, cov_diag)
    return -dist.log_prob(target).mean()

if i_test==1:
    target = torch.randn(10, 2)
    loss = NLLLoss(output_2, target, second_output_size)
    print(loss)

In [78]:
x_data_n=np.array(x_data_unpacked)
y_data_n=np.array(y_data_unpacked)
print(x_data_n.shape)
print(y_data_n.shape)
x_data_n=x_data_n
y_data_n=y_data_n
n_ens=5
n_all=len(x_data_n)
n_all_sub=n_all//n_ens

x_data_t=torch.tensor(x_data_n/25.0,dtype=torch.float32)
y_data_t=torch.tensor(y_data_n,dtype=torch.float32)
x_pia_data_t=torch.tensor(x_pia_data,dtype=torch.float32)
y_pia_data_t=torch.tensor(y_pia_data,dtype=torch.float32)
print(x_data_t.shape)
n_seq_data_t=torch.tensor(n_seq_data,dtype=torch.int32)
p_type_data=np.array(p_type_data)
a=np.nonzero(p_type_data>0)[0]


(254344, 72, 2)
(254344, 72, 2)
torch.Size([254344, 72, 2])


In [85]:
print(p_type_data[a].shape)
a=np.nonzero(p_type_data==2)[0]
ns1=1
n_ens=3
#print(n_seq_data)
x_data_strat_n=torch.tensor(x_data_n[a][::ns1],dtype=torch.float32)
y_data_strat_n=torch.tensor(y_data_n[a][::ns1],dtype=torch.float32)
x_pia_data_strat=torch.tensor(np.array(x_pia_data)[a][::ns1],dtype=torch.float32)
y_pia_data_strat=torch.tensor(np.array(y_pia_data)[a][::ns1],dtype=torch.float32)
n_seq_data_strat=torch.tensor(np.array(n_seq_data)[a][::ns1],dtype=torch.int32)

strat_dataL=[]
strat_loaderL=[]
n_all_str=x_data_strat_n.shape[0]
print(n_all_str)
n_all_sub=n_all_str//n_ens
print(n_all_sub)

(34551,)
34551
11517


In [86]:

for i in tqdm.tqdm(range(n_ens)):
    strat_data=torch.utils.data.TensorDataset(x_data_strat_n[i*n_all_sub:(i+1)*n_all_sub,:,:],y_data_strat_n[i*n_all_sub:(i+1)*n_all_sub,:,:],n_seq_data_strat[i*n_all_sub:(i+1)*n_all_sub],x_pia_data_strat[i*n_all_sub:(i+1)*n_all_sub,:],y_pia_data_strat[i*n_all_sub:(i+1)*n_all_sub,:])
    print(i,strat_data.tensors[0].shape)
    strat_loader=torch.utils.data.DataLoader(strat_data,batch_size=64,shuffle=True)
    strat_dataL.append(strat_data)
    strat_loaderL.append(strat_loader)
    #print(i,strat_data.tensors[0].shape)


100%|██████████| 3/3 [00:00<00:00, 196.99it/s]

0 torch.Size([11517, 72, 2])
1 torch.Size([11517, 72, 2])
2 torch.Size([11517, 72, 2])


In [87]:
n_epoch=15
model_list=[]
#
lambda1=0.5
n_ens3=3
input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp=2,1,64,2,2,3,2
for i in tqdm.tqdm(range(n_ens3)):
    model_list.append(LSTMPackedModel(input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp))
    model_strat=model_list[i]#
    strat_criterion=nn.MSELoss(reduction='none')
    strat_optimizer=torch.optim.Adam(model_strat.parameters(),lr=0.001)

    for epoch in range(n_epoch):
        running_loss=0
        for iloader in range(n_ens3):
            for x,y,n_seq, x_sec, y_sec in strat_loaderL[iloader]:
                strat_optimizer.zero_grad()
                y_pred1,y_pred2=model_strat(x,n_seq,x_sec)
                n_seq1=y_pred1.shape[1]
       
                loss=strat_criterion(y_pred1,y[:,:n_seq1,:])
                mask=torch.arange(n_seq1).expand(y.shape[0],n_seq1)<n_seq.unsqueeze(1)
        
                masked_loss=(loss*mask.unsqueeze(2)).sum()
                masked_loss=masked_loss/mask.sum()
                total_loss=masked_loss+lambda1*NLLLoss(y_pred2,y_sec,second_output_size)
                total_loss.backward()
                running_loss+=total_loss.item()
                strat_optimizer.step()
        print(epoch,running_loss/len(strat_loader))
    torch.onnx.export(model_list[i],(x[:1][:,:,:],n_seq[:1],x_sec[:1]),'conv_ku_densi_net_land_feb16_2025_%2.2i.onnx'%i,export_params=True,opset_version=11,do_constant_folding=True,input_names=['input_1','n_seq','input_2'],output_names=['output1','output2'])


  0%|          | 0/3 [00:00<?, ?it/s]

0 0.898073322698474
1 -2.339838701941901
2 -2.7656547717957034
3 -2.8127212063926788
4 -2.876323032565415
5 -2.9760439957181615
6 -3.0484896474000482
7 -3.089240730388297
8 -3.1189644364847076
9 -3.141674852805833
10 -3.1761795971542597
11 -3.175427285167906
12 -3.213492615169121
13 -3.235259349292351
14 -3.2562927286554544


/var/folders/x_/d2_jzyq50052xh1_tk02bnmc0000gq/T/ipykernel_4150/1131222009.py:23: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  n_batch = len(lengths)
[W218 10:41:17.947657000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W218 10:41:17.950243000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size o

0 0.7537125536137157
1 -2.2442228705104856
2 -2.680083980369899
3 -2.8551936354695093
4 -2.8549209251999854
5 -2.9626000865466064
6 -3.0339454938761063
7 -3.0716992757179673
8 -3.026126131291191
9 -3.143761848948068
10 -3.1321229713865453
11 -3.184983081743121
12 -3.199439244489703
13 -3.2332118831160996
14 -3.2327727860874598


[W218 10:49:41.512705000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W218 10:49:41.513880000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W218 10:49:41.662170000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PadPacked type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
 67%|██████▋   | 2/3 [16:59<08:28, 508.75s/it]

0 0.6282210427853796
1 -2.4004403583705427
2 -2.655899199160437
3 -2.833868743975957
4 -2.9069929643430643
5 -2.983242106851604
6 -3.0062335507944224
7 -3.0825338578265575
8 -3.1063892021568287
9 -3.132279234048393
10 -3.1200731791555882
11 -3.189896329699291
12 -3.189248254357113
13 -3.177522522976829
14 -3.220627219809426


[W218 10:58:03.911037000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W218 10:58:03.913234000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W218 10:58:03.060772000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PadPacked type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
100%|██████████| 3/3 [25:22<00:00, 507.37s/it]


In [68]:
for i in tqdm.tqdm(range(n_ens3)):
    torch.onnx.export(model_list[i],(x[:1][:,:,:],n_seq[:1],x_sec[:1]),'conv_ku_densi_net_feb16_2025_%2.2i.onnx'%i,export_params=True,opset_version=11,do_constant_folding=True,input_names=['input_1','n_seq','input_2'],output_names=['output1','output2'])

  0%|          | 0/3 [00:00<?, ?it/s]/var/folders/x_/d2_jzyq50052xh1_tk02bnmc0000gq/T/ipykernel_4150/1131222009.py:23: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  n_batch = len(lengths)
[W216 16:48:33.228141000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W216 16:48:33.229215000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exportin

In [88]:
y_pred1,y_pred2=model_list[0](x_data_strat_n,n_seq_data_strat,x_pia_data_strat)
y_pred1_np=y_pred1.detach().numpy()
y_pred2_np=y_pred2.detach().numpy()
y_test_np=y_data_strat_n.detach().numpy()
y_pia_test_np=y_pia_data_strat.detach().numpy()

In [89]:
print(np.corrcoef(y_pred2_np[:,1],y_pia_test_np[:,1]))
print(np.corrcoef(y_pred2_np[:,0],y_pia_test_np[:,0]))

[[1.         0.94791359]
 [0.94791359 1.        ]]
[[1.        0.9580755]
 [0.9580755 1.       ]]


In [91]:
print(x_pia_data_strat.max(axis=0))

torch.return_types.max(
values=tensor([18.]),
indices=tensor([11472]))


In [65]:

for epoch in range(n_epoch2):
    running_loss=0
    for x,y,n_seq in strat_loader:
        strat_ku_optimizer.zero_grad()
        y_pred=model_ku_strat(x[:,:,a_feat],n_seq)
        n_seq1=y_pred.shape[1]
       
        loss=strat_criterion(y_pred,y[:,:n_seq1,:2])
        mask=torch.arange(n_seq1).expand(y.shape[0],n_seq1)<n_seq.unsqueeze(1)
        
        masked_loss=(loss*mask.unsqueeze(2)).sum()
        masked_loss=masked_loss/mask.sum()
    
        masked_loss.backward()
        running_loss+=masked_loss.item()
        strat_ku_optimizer.step()
    print(epoch,running_loss/len(strat_loader))

0 0.03449178049541849
1 0.03447961658284032
2 0.03437462921881919
3 0.034343278710004525
4 0.03427321770467418
5 0.03410242947836896
6 0.03394827105069189
7 0.03378847528615301
8 0.033944862818778704
9 0.03381727487563515


In [60]:
for i in range(1):
    torch.onnx.export(model_list[i],(x[:1][:,:,:],n_seq[:1],x_sec[:1]),'conv_ku_densi_net_feb16_2025_%2.2i.onnx'%i,export_params=True,opset_version=11,do_constant_folding=True,input_names=['input_1','n_seq','input_2'],output_names=['output'])

/var/folders/x_/d2_jzyq50052xh1_tk02bnmc0000gq/T/ipykernel_4150/1131222009.py:23: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  n_batch = len(lengths)
[W216 15:54:27.288979000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W216 15:54:27.297769000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size o

In [64]:
print(n_seq_data_strat.max())

tensor(72, dtype=torch.int32)


In [55]:
model_strat.eval()
output=model_strat(x_data_strat_n[::2,:,:],n_seq_data_strat[::2])

In [58]:
output_np=output.detach().numpy()
import pickle
pickle_out={"output":output_np,"y_data":y_data_strat_n[::2,:,:].detach().numpy(),"x_data":x_data_strat_n[::2,:,:].detach().numpy(),"n_seq":n_seq_data_strat[::2].detach().numpy(),"z_meas":z_meas2_strat[::2],"bin_nodes":bin_nodesL_strat[::2]}
pickle.dump(pickle_out,open("output_strat_jan28_2025.pkl","wb"))

In [70]:
torch.onnx.export(model_ku_strat,(x[:1][:,:,a_feat],n_seq[:1]),'conv_and_strat_model_ku_jan28_2025.onnx',export_params=True,opset_version=11,do_constant_folding=True,input_names=['input','n_seq'],output_names=['output'])

[W129 11:23:13.674287000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W129 11:23:13.675937000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
[W129 11:23:13.885182000 shape_type_inference.cpp:1999] Warning: The shape 